In [1]:
search_for = 129
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.07492876052856445
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 423524321
type: [1, 1, 1, 1, 129] 129
cases of this type: 276922881
100000 13.93267960472153
200000 23.11532535969467
300000 28.078819932081224
400000 29.666095437948638
500000 33.03617777646053
600000 38.01289573454902
700000 37.21502123618198
800000 36.840823322237384
900000 31.596125001038676
1000000 35.92148449158236
1100000 16.360843985626005
1200000 33.70327832409662
1300000 34.30547805110885
1400000 33.580511691019765
1500000 37.743397689016845
1600000 38.45778503096406
1700000 34.16560419861131
1800000 36.647914063192566
1900000 36.9488987713759
2000000 38.77002025548248
2100000 33.40640497520042
2200000 8.035380613892062
2300000 37.83787408049121
2400000 37.76136602520807
2500000 39.105991178803315
2600000 39.62411223307592
2700000 35.88317153794024
2800000 38.41736413868176
2900000 35.488826027245
3000000 38.39875909299

29500000 28.619312115464787
29600000 32.90783063838649
29700000 29.27499985637948
29800000 27.92863405357571
29900000 18.062740270487478
30000000 28.551608863110832
30100000 34.08091640219719
30200000 29.049017509081462
30300000 20.10865025771601
30400000 24.563137350892042
30500000 27.638805377409497
30600000 32.23897277278558
30700000 28.870840418913843
30800000 30.680588828529356
30900000 27.79568835358999
31000000 26.75776261694982
31100000 26.522333800969882
31200000 13.134467890066219
31300000 27.02611886662852
31400000 28.43164879974262
31500000 26.457045690474963
31600000 31.605097325540566
31700000 27.47858949023888
31800000 30.78453244241653
31900000 28.730711779398554
32000000 12.85722733380622
32100000 33.15428247283692
32200000 29.133159130029654
32300000 32.10016267313193
32400000 29.629339601761394
32500000 11.586275954964236
32600000 33.072030664124775
32700000 28.064866385253765
32800000 30.599437951023287
32900000 29.012744796068155
33000000 28.344171070250802
3310000

59200000 26.034403654338483
59300000 26.66297972649823
59400000 25.19546868917285
59500000 27.44586608957111
59600000 27.329120502353046
59700000 11.991556723207175
59800000 29.503742838301292
59900000 28.57916832257106
60000000 31.545605965194387
60100000 28.312743788640798
60200000 30.563267806050618
60300000 28.00047811036893
60400000 29.93579350721051
60500000 27.71608040075424
60600000 15.74260195567519
60700000 24.679295678702132
60800000 25.32713475393253
60900000 27.827528136386732
61000000 25.451126691532693
61100000 25.037416934188684
61200000 14.331567643314928
61300000 21.992484853716274
61400000 25.814165606115886
61500000 26.36056657529348
61600000 24.6666867376543
61700000 28.216715334752482
61800000 14.71048039620433
61900000 23.730821540979914
62000000 30.254480790596862
62100000 28.079753239871877
62200000 31.30548890642418
62300000 27.40163406181492
62400000 30.392288895395293
62500000 26.86906411888355
62600000 29.375302076703264
62700000 27.022160985370153
62800000

88900000 22.258401887842886
89000000 23.36132248481619
89100000 14.286403435552677
89200000 18.783028168464494
89300000 22.659921769721702
89400000 24.381596579516483
89500000 11.815586862009429
89600000 21.377493100634453
89700000 24.34757779726896
89800000 23.575880795677584
89900000 25.412014981266665
90000000 26.222039293150953
90100000 25.899890834181893
90200000 24.618868862968682
90300000 25.624044825401498
90400000 23.744987301612987
90500000 25.535681196511124
90600000 23.45733622856304
90700000 22.246736399033743
90800000 23.151662721113865
90900000 10.840027425707518
91000000 21.199843738478524
91100000 23.677411699795332
91200000 21.304291531664095
91300000 8.718480313334323
91400000 22.179107833672077
91500000 22.255388824548586
91600000 15.35240187529992
91700000 18.107625959917744
91800000 21.47626890641072
91900000 21.34476668214689
92000000 24.716331273400098
92100000 22.44092392427857
92200000 26.02809940535303
92300000 23.666352736053625
92400000 24.741496513118655
9

118000000 23.762830552455572
118100000 23.2971162291419
118200000 22.690886133739287
118300000 22.337384127793435
118400000 20.005209359449356
118500000 21.151353807976008
118600000 20.33410181515699
118700000 20.56712341990261
118800000 20.657332044115687
118900000 20.062563406025475
119000000 13.839769212622889
119100000 17.463388144533237
119200000 7.093262107721443
119300000 15.68026627801316
119400000 15.930419520736667
119500000 19.179539108716952
119600000 22.09189562260497
119700000 19.073650444521025
119800000 21.866470981530515
119900000 20.36120573162614
120000000 19.84417197513743
120100000 23.456477875712462
120200000 21.30167112798136
120300000 24.775214051120496
120400000 21.36758730670199
120500000 20.810971335613758
120600000 21.778222852768273
120700000 19.29004014278165
120800000 22.2770498742007
120900000 19.184498102829057
121000000 21.072538896857846
121100000 19.79772105659973
121200000 8.280905988496999
121300000 13.114951963839246
121400000 15.954123134068544
1

...

290200000 4.532940297970043
290300000 5.2291321966335484
290400000 5.053356840624544
290500000 5.186586325484969
290600000 5.083907274836379
290700000 4.7121871234437585
290800000 3.56267624528337
290900000 1.1116327106093031
291000000 1.125840843661966
291100000 5.9399785909842375
291200000 4.699090726178542
291300000 5.149655614810937
291400000 5.147334273701383
291500000 5.056393471628218
291600000 4.819271104797654
291700000 4.733006479312224
291800000 5.184176211439582
291900000 5.0069287874597075
292000000 6.110162827544718
292100000 5.234468709068447
292200000 5.209818083997958
292300000 5.324177096504188
292400000 4.617889844648934
292500000 5.128162536856552
292600000 4.859849561395948
292700000 5.13925767645805
292800000 4.806183451355358
292900000 5.101581355438733
293000000 1.1018501698643466
293100000 1.1112640843209525
293200000 3.897824734704489
293300000 4.5971136818252925
293400000 5.1775821665662
293500000 5.022908421029771
293600000 4.806287856888194
293700000 4.6246

319300000 4.183180153612113
319400000 4.08028379085018
319500000 3.9395221292636813
319600000 3.8749147431327144
319700000 4.149743204160245
319800000 4.160031349437873
319900000 5.272546722057002
320000000 5.35675095961132
320100000 4.619767873073691
320200000 4.4025663381843385
320300000 4.148542119433558
320400000 4.54934782371797
320500000 4.619097946840538
320600000 4.538942972285909
320700000 3.9964404094958885
320800000 5.241125040787071
320900000 1.2468575190657736
321000000 0.8507801912855387
321100000 2.7293311536538263
321200000 4.759728202641804
321300000 3.937086744066819
321400000 4.49316412223549
321500000 4.430873613489888
321600000 4.348383606741939
321700000 3.988388617347082
321800000 4.4474412421574785
321900000 4.527702536559426
322000000 4.770450056944608
322100000 6.078991704823308
322200000 4.804753775157401
322300000 4.788697846933177
322400000 4.9067520447521025
322500000 4.856553060361943
322600000 4.7992579313812
322700000 4.920448974728411
322800000 4.60262

348100000 1.7163278529958785
348200000 2.1295056709723057
348300000 1.1964206139983735
348400000 1.5735512429858212
348500000 1.2518369712872484
348600000 1.23913375843775
348700000 1.285144946607413
348800000 0.5007266754908501
348900000 0.4237129942588634
349000000 0.8157914592079083
349100000 1.7780499593833328
349200000 1.2240090785694269
349300000 1.2500584142739555
349400000 1.2993313722036341
349500000 1.3519056536508376
349600000 1.2900160844159172
349700000 1.3130542491681476
349800000 1.2952221332462848
349900000 1.5870605825608386
350000000 1.9794791878701565
350100000 1.286106660504714
350200000 1.2688021999443684
350300000 1.2703637096185743
350400000 1.135009597637664
350500000 1.663178748109214
350600000 1.4754775926667871
350700000 1.4684840836311381
350800000 1.921341969624404
350900000 0.9679697460131169
351000000 0.31743454863177206
351100000 0.31798516801821
351200000 1.2924119308040818
351300000 1.1815153287686864
351400000 1.531406084867421
351500000 1.25469404493

376100000 0.16450841763266325
376200000 0.16425320580181216
376300000 0.1642261673976607
376400000 0.16383942949673533
376500000 0.1635980455915716
376600000 0.22010477879093687
376700000 0.16409760635937254
376800000 0.16299081211208832
376900000 0.16259288092679647
377000000 0.16178283629774454
377100000 0.1577203824002544
377200000 0.15955075219458012
377300000 0.16001813078725535
377400000 0.1584717950716906
377500000 0.1595765103476637
377600000 0.21774191749389568
377700000 0.16529756294100284
377800000 0.1650953691645245
best so far: 0
type: [1, 1, 3, 1, 43] 129
cases of this type: 30769209
377900000 0.9268891467703521
378000000 0.9388471584173684
378100000 0.9035687025769544
378200000 1.1805106275551598
378300000 0.9065317718800961
378400000 0.9406193677615808
378500000 0.9080376152443805
378600000 1.1068118513073697
378700000 0.8566571691739976
378800000 0.8534964868496239
378900000 1.4885407318704058
379000000 1.9084981680507747
379100000 0.8300716951970498
379200000 0.808862

404100000 0.6541592995490154
404200000 0.7742239896245923
404300000 0.7086417228663684
404400000 0.7518790859182198
404500000 0.7231888500332594
404600000 0.6490875313739876
404700000 0.8042028431170213
404800000 0.659596150944594
404900000 0.7128231685789644
405000000 0.7559393903298954
405100000 0.7576129972359896
405200000 0.7208262893917066
405300000 0.6387604611288991
405400000 0.8037830560676986
405500000 0.673553696959561
405600000 0.6746939029407958
405700000 0.7607658643633141
405800000 0.7263794385036231
405900000 0.6886656699290236
406000000 0.618133731271098
406100000 0.7227035317100268
406200000 0.6819889949749908
406300000 0.6506378421948393
406400000 0.7062566159403921
406500000 0.7114211631211115
406600000 0.6340309521508217
406700000 0.5910094955096841
406800000 0.64045619129287
406900000 0.6886156224043887
407000000 0.6057454449738939
407100000 0.6145938926570693
407200000 0.7626857387917857
407300000 0.6618299424983065
407400000 0.6201379635330935
407500000 0.6491430